# Тестовое

In [38]:
import pandas as pd
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

from IPython.display import FileLink

from sqlalchemy import text, create_engine

# Укажите путь к вашему CSV/Excel/JSON файлу
file_path = 'C:\\Users\\Пользователь\\OneDrive\\Рабочий стол\\data.csv'

# Загрузите данные из файла в объект DataFrame
df = pd.read_csv(file_path)

# Укажите параметры подключения к вашей базе данных PostgreSQL
username = 'postgres'
password = 'Vbkkbfhlth2022'
host = 'localhost'
port = '5432'
database = 'postgres'

# Укажите путь к вашему файлу сертификата SSL
ssl_cert = 'path_to_your_ssl_certificate.pem'

# Формируем строку подключения с параметрами SSL и тайм-аутом
connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'

# Создаем соединение
engine = create_engine(connection_string)
con=engine.connect()

# Пример запроса SQL и его выполнение
sql_query = "SELECT * FROM public.test"
df = pd.read_sql(sql_query, engine)

# Вывод результата
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Склад           2050 non-null   object 
 1   Дата            2050 non-null   object 
 2   Продажи рублей  2050 non-null   float64
dtypes: float64(1), object(2)
memory usage: 48.2+ KB


,Склад,Дата,Продажи рублей
0,Склад 1,2013-01-08,1776309.00
1,Склад 1,2013-01-09,1670458.80
2,Склад 1,2013-01-13,2065636.40
3,Склад 1,2013-01-14,1736802.80
4,Склад 1,2013-01-15,1689900.80
...,...,...,...
2045,Склад 66,2015-12-20,922448.00
2046,Склад 66,2015-12-23,958763.80
2047,Склад 66,2015-12-27,1072242.50
2048,Склад 66,2015-12-28,1222390.70


## 1.Отчёт по дням

Вариант решения через **FULL JOIN** и **INTERVAL**, чтобы сохранились все строки и отображались данные по полям *Склад* и *Дата* использовали функцию **COALESCE**.

In [119]:
query = """
SELECT COALESCE(t1.Склад, t2.Склад, t3.Склад) Склад, COALESCE(t1.Дата, t2.Дата + INTERVAL '1 year', t3.Дата + INTERVAL '1 month') Дата, 
    t1."Продажи рублей"/1000 AS "Продажи, тыс.руб.",
    t2."Продажи рублей"/1000 AS "Продажи предыдущего года, тыс.руб.",
    t3."Продажи рублей"/1000 AS "Продажи предыдущего месяца, тыс.руб."
FROM 
    public.test t1
FULL JOIN
    public.test t2 ON t1.Склад = t2.Склад AND  t2.Дата = t1.Дата - INTERVAL '1 year'
FULL JOIN
    public.test t3 ON t1.Склад = t3.Склад AND  t3.Дата = t1.Дата - INTERVAL '1 month'
WHERE t1."Продажи рублей" IS NOT NULL OR t2."Продажи рублей" IS NOT NULL OR t3."Продажи рублей" IS NOT NULL
ORDER BY Дата, Склад;
"""
df1 = pd.io.sql.read_sql(sql=text(query), con = con)
df1.info()
df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5068 entries, 0 to 5067
Data columns (total 5 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Склад                               5068 non-null   object        
 1   Дата                                5068 non-null   datetime64[ns]
 2   Продажи, тыс.руб.                   2050 non-null   float64       
 3   Продажи предыдущего года, тыс.руб.  2050 non-null   float64       
 4   Продажи предыдущего месяца, тыс.ру  2086 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 198.1+ KB


,Склад,Дата,"Продажи, тыс.руб.","Продажи предыдущего года, тыс.руб.","Продажи предыдущего месяца, тыс.ру"
0,Склад 206,2013-01-01,416.54,NaN,NaN
1,Склад 23,2013-01-01,2415.13,NaN,NaN
2,Склад 66,2013-01-02,1067.76,NaN,NaN
3,Склад 143,2013-01-03,1551.47,NaN,NaN
4,Склад 206,2013-01-03,970.80,NaN,NaN
...,...,...,...,...,...
5063,Склад 143,2016-12-28,NaN,1341.61,NaN
5064,Склад 66,2016-12-28,NaN,1222.39,NaN
5065,Склад 1,2016-12-29,NaN,1675.48,NaN
5066,Склад 227,2016-12-31,NaN,1707.51,NaN


In [120]:
df1.to_excel('Отчёт_суточный.xlsx', index=False)

excel_filename = 'Отчёт_суточный.xlsx'

FileLink(excel_filename)

C:\Windows\system32\Отчёт_суточный.xlsx

Вариант с RIGHT JOIN не имеет смысла, т.к. у нас есть строки, где в полях продаж в аналогичный период предыдущего месяца и года, не везде имеют данные одновременно. 

## 2.Отчёт помесячный

In [121]:
query = """
SELECT t1.Склад, EXTRACT(YEAR from t1.Месяц) Год,  EXTRACT(MONTH from t1.Месяц) Месяц, t1."Продажи, тыс.руб.",
    t2."Продажи, тыс.руб." AS "Продажи предыдущего года, тыс.руб.",
    t3."Продажи, тыс.руб." AS "Продажи предыдущего месяца, тыс.руб."
FROM 
    (SELECT Склад, DATE_TRUNC('month', Дата) Месяц, SUM("Продажи рублей"/1000) "Продажи, тыс.руб."
    FROM public.test
    GROUP BY Склад, DATE_TRUNC('month', Дата)) t1
LEFT JOIN
    (SELECT Склад, DATE_TRUNC('month', Дата) Месяц, SUM("Продажи рублей"/1000) "Продажи, тыс.руб."
    FROM public.test
    GROUP BY Склад, DATE_TRUNC('month', Дата)) t2 ON t1.Склад = t2.Склад AND t2.Месяц = t1.Месяц - INTERVAL '1 year'
LEFT JOIN
    (SELECT Склад, DATE_TRUNC('month', Дата) Месяц, SUM("Продажи рублей"/1000) "Продажи, тыс.руб."
    FROM public.test
    GROUP BY Склад, DATE_TRUNC('month', Дата)) t3 ON t1.Склад = t3.Склад AND t3.Месяц = t1.Месяц - INTERVAL '1 month'
ORDER BY Год, Месяц;
"""

# Выполняем запрос и загружаем результат в DataFrame
df2 = pd.read_sql(query, con=engine)

# Выводим DataFrame
df2.info()
df2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Склад                               252 non-null    object 
 1   Год                                 252 non-null    float64
 2   Месяц                               252 non-null    float64
 3   Продажи, тыс.руб.                   252 non-null    float64
 4   Продажи предыдущего года, тыс.руб.  168 non-null    float64
 5   Продажи предыдущего месяца, тыс.ру  245 non-null    float64
dtypes: float64(5), object(1)
memory usage: 11.9+ KB


,Склад,Год,Месяц,"Продажи, тыс.руб.","Продажи предыдущего года, тыс.руб.","Продажи предыдущего месяца, тыс.ру"
0,Склад 51,2013.00,1.00,5911.29,NaN,NaN
1,Склад 66,2013.00,1.00,7150.41,NaN,NaN
2,Склад 206,2013.00,1.00,7177.73,NaN,NaN
3,Склад 227,2013.00,1.00,6054.82,NaN,NaN
4,Склад 143,2013.00,1.00,11753.20,NaN,NaN
5,Склад 1,2013.00,1.00,14498.91,NaN,NaN
6,Склад 23,2013.00,1.00,20434.49,NaN,NaN
7,Склад 1,2013.00,2.00,11704.46,NaN,14498.91
8,Склад 23,2013.00,2.00,5841.94,NaN,20434.49
9,Склад 206,2013.00,2.00,4036.00,NaN,7177.73


In [122]:
df3.to_excel('Отчёт_помесячный.xlsx', index=False)

excel_filename = 'Отчёт_помесячный.xlsx'

FileLink(excel_filename)

C:\Windows\system32\Отчёт_помесячный.xlsx

Объеденим две полученных таблицы и первоначальную в один файл

In [123]:
output_file = 'MergedExcel.xlsx'
with pd.ExcelWriter(output_file) as writer:
    # Запишите каждую таблицу в свой лист
    df.to_excel(writer, sheet_name='Исходные данные', index=False)
    df1.to_excel(writer, sheet_name='Посуточный отчет', index=False)
    df2.to_excel(writer, sheet_name='Помесячный отчет', index=False)

excel_filename = 'MergedExcel.xlsx'

FileLink(excel_filename)

C:\Windows\system32\MergedExcel.xlsx